In [1]:

import pandas as pd
import numpy as np

from axisfuzzy.analysis.app.model import Model
from axisfuzzy.analysis.accessor import FuzzyAccessor
from axisfuzzy.analysis.component.basic import (
    ToolNormalization,
    ToolFuzzification,
    ToolSimpleAggregation
)
from axisfuzzy.analysis.build_in import ContractCrispTable, ContractFuzzyTable
from axisfuzzy.fuzzifier import Fuzzifier

In [2]:
my_fuzzifier = Fuzzifier(
    mf='gaussmf',
    mtype='qrofn',  # 假设目标是 q-rung orthopair fuzzy number
    pi = 0.2,
    mf_params=[{'sigma': 0.15, 'c': 0.5}]
)

In [3]:
# --- 子模型：数据预处理流程 ---
class DataPreprocessingModel(Model):
    def get_config(self) -> dict:
        pass

    def __init__(self, norm_method: str = 'min_max'):
        super().__init__() # __init__ 现在只做初始化
        self.normalizer = ToolNormalization(method=norm_method, axis=0)
        self.fuzzifier = ToolFuzzification(fuzzifier=my_fuzzifier)

    def forward(self, data: ContractCrispTable) -> ContractFuzzyTable:
        norm_data = self.normalizer(data)
        # fuzzy_data = self.fuzzifier(norm_data)
        return norm_data

In [4]:
class MainAnalysisModel(Model):
    def get_config(self) -> dict:
        pass

    def __init__(self):
        super().__init__() # __init__ 现在只做初始化
        self.preprocessing_pipeline = DataPreprocessingModel(norm_method='z_score')
        self.aggregator = ToolSimpleAggregation(operation='mean', axis=1)

    def forward(self, raw_input_data: ContractCrispTable):
        processed_data = self.preprocessing_pipeline(raw_input_data)
        final_scores = self.aggregator(processed_data)
        return final_scores

In [5]:
df = pd.DataFrame(np.random.rand(5, 4), columns=[f'Feature_{i}' for i in range(4)])

In [6]:
main_model = MainAnalysisModel()

In [10]:
df.fuzzy.run(main_model, return_intermediate=True)

(0    0.049980
 1   -0.532458
 2    0.715137
 3   -0.115993
 4   -0.116666
 dtype: float64,
 {'input_raw_input_data_a6250ad1':    Feature_0  Feature_1  Feature_2  Feature_3
  0   0.761334   0.406876   0.100124   0.778195
  1   0.078278   0.926545   0.113000   0.033100
  2   0.582740   0.718376   0.562442   0.840895
  3   0.246043   0.776010   0.382364   0.206172
  4   0.708315   0.210631   0.556662   0.057736,
  'DataPreprocessingModel_38d67372':    Feature_0  Feature_1  Feature_2  Feature_3
  0   0.956179  -0.688542  -1.066663   0.998945
  1  -1.327535   1.093296  -1.010095  -0.885498
  2   0.359073   0.379528   0.964426   1.157521
  3  -0.766634   0.577144   0.173294  -0.447777
  4   0.778917  -1.361425   0.939037  -0.823191,
  'ToolSimpleAggregation_run_782a3d25': 0    0.049980
  1   -0.532458
  2    0.715137
  3   -0.115993
  4   -0.116666
  dtype: float64})

In [ ]:
main_model = MainAnalysisModel()
main_model.build()

In [ ]:
main_model.summary()

In [ ]:
main_model.visualize()

In [ ]:
main_model.run(df)